In [1]:
#data wrangling. now that we have the numbers, it's just a matter of
#combining them into a useful output

In [18]:
import tempfile
from qiime2 import Visualization
import pandas as pd

In [10]:
working_dir = '/home/dylan/Documents/june_reset'

In [8]:
compartments = ['M', 'T', 'S']
references = ['greengenes', 'silva', 'greengenes_metaxa2',
              'silva_metaxa2']

In [118]:
proportion_unassigned = []
references1 = []
compartments1 = []
proportion_mitochondria = []
references5 = []
compartments5 = []
for reference in references:
    for compartment in compartments:
        with tempfile.TemporaryDirectory() as temp_dir:
            dataset = compartment + '_' + reference
            tbp_path = working_dir + '/output/' + dataset + '_tbp.qzv'
            tbp = Visualization.load(tbp_path)
            tbp.export_data(temp_dir)
            df1 = pd.read_csv(temp_dir + '/level-1.csv')
            df1['total'] = df1.sum(axis = 1)
            df1['compartment'] = compartment
            df1['reference'] = reference
            proportion_unassigned.extend(list(df1['Unassigned'] / df1['total']))
            references1.extend(list(df1['reference']))
            compartments1.extend(df1['compartment'])
            df5 = pd.read_csv(temp_dir + '/level-5.csv')
            df5['total'] = df5.sum(axis = 1)
            df5['compartment'] = compartment
            df5['reference'] = reference
            if 'greengenes' in reference:
                proportion_mitochondria.extend(list(df5['k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rickettsiales;f__mitochondria'] / df5['total']))
            elif 'silva' in reference:
                proportion_mitochondria.extend(list(df5['D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rickettsiales;D_4__Mitochondria'] / df5['total']))
            references5.extend(list(df5['reference']))
            compartments5.extend(df5['compartment'])

In [124]:
lvl1 = list(zip(proportion_unassigned, references1, compartments1))
lvl1_df = pd.DataFrame(lvl1, columns = ['proportion unassigned', 'reference taxonomy', 'tissue compartment'])
lvl1_df.to_csv(working_dir + '/output/lvl1.csv')

In [125]:
lvl5 = list(zip(proportion_mitochondria, references5, compartments5))
lvl5_df = pd.DataFrame(lvl5, columns = ['proportion mitochondria', 'reference taxonomy', 'tissue compartment'])
lvl5_df.to_csv(working_dir + '/output/lvl5.csv')